# Group assignment 1.2: Numerical integration and Taylor series approximations

*[CEGM1000 MUDE](http://mude.citg.tudelft.nl/)*

*Written by: Anna Störiko, Ronald Brinkgeve*

*Due: Friday, September 12, 2025*

## Part 1: Discharge estimation and numerical integration

In many hydrological and engineering applications, it is important to know how much water flows through a river or stream.
In small rivers, a simple method to estimate the discharge is so called dilution gauging.
In this method, a known amount of salt is added to the river and the concentration of the salt is measured downstream.

<center><img src="https://files.mude.citg.tudelft.nl/dilution_gauging.svg" width="50%"/></center>

Based on a mass balance, the discharge $Q$ [$\mathrm{L}^3 \mathrm{T}^{-1}$] can then be estimated from the injected mass $m [\mathrm{M}]$ and the area under the concentration curve $A$:

$$Q = \frac{m}{A} = \frac{m}{\int_0^{t_{\text{end}}} c(t) dt}$$

where $c(t) [\mathrm{M} \mathrm{L}^{-3}]$ is the concentration of salt in the river above the baseline level at time $t$ and $t_{\text{end}}$ is the end time of the measurement, well after the concentration returned to its baseline value.

The plot below shows such a concentration curve. In this assignment, you will determine the discharge based on this curve.
To evaluate the integral under the curve from the discrete measurements, you will have to use numerical integration techniques.

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# time in seconds
time = np.array([  0.,  10.,  20.,  30.,  41.,  50.,  60.,  70.,  78.,  90., 100.,
       110., 120., 130., 140., 150., 170., 180., 190., 200.])
# concentrations in g/L
concentration = np.array([0, 0.02, 0.1, 0.4, 0.7, 0.6, 0.4, 0.3, 0.22,
        0.18, 0.15, 0.12, 0.1, 0.08, 0.06, 0.04, 0.01, 0, 0, 0])
plt.plot(time, concentration, marker="o")
plt.ylabel("Concentration [g/L]")
plt.xlabel("Time [s]")

### Numerical integration of the concentration curve

<div style="background-color:#AABAB2; color: black; width:90%; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px">
<p>

**Task 1.1**

Compute the area under the concentration curve by numerical integration. Compare two numerical integration methods of your choice.

</p>
</div>

In [ ]:
# Compute the time difference between two consecutive points
dt = np.diff(time)

### YOUR CODE LINES HERE

In [ ]:
# Compute the time difference between two consecutive points
dt = np.diff(time)

# Compute the area of the concentration curve with the trapezoidal rule
trapezoid_areas = (concentration[:-1] + concentration[1:]) / 2 * dt
total_area_trapz = np.sum(trapezoid_areas)

# Compute the area of the concentration curve with the left Riemann sum
total_area_left_riemann = np.sum(concentration[:-1] * dt)

# Compute the area of the concentration curve with the right Riemann sum
total_area_right_riemann = np.sum(concentration[1:] * dt)

print(
f"""
Total area under the concentration curve:
Left Riemann sum: {total_area_left_riemann:.2f} g/L s
Right Riemann sum: {total_area_right_riemann:.2f} g/L s
Trapezoidal rule: {total_area_trapz:.2f} g/L s
"""
)

<div style="background-color:#AABAB2; color: black; width:90%; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px">
<p>

**Task 1.2**

Which integration method is best suited for this dataset? Why?

</p>
</div>

<div style="background-color:#FAE99E; color: black; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px; width: 90%">

**Solution 1.2**

For the dataset, the trapezoidal rule is suitable integration technique.
For a given number of integration points, it (theoretically) has a lower error compared to the left and right Riemann sum.

While Simpson's rule would have an even lower error, it requires equally spaced measurement intervals which are not available for this data set.
The error of the midpoint rule scales the same with the number of integration intervals as for the trapezoidal rule, but it requires evaluating the function at the midpoint of each interval. This is not possible with fixed measurements.
</p></div>

<div style="background-color:#AABAB2; color: black; width:90%; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px">
<p>

**Task 1.3**

Compute the discharge based on the concentration data. The amounf of salt used tracer test is 100 g.

</p>
</div>

In [ ]:
### YOUR CODE LINES HERE

In [ ]:
mass = 200 # g
discharge = mass / total_area_trapz # g / (g/L * s)
print(f"{discharge:.2f} L/s")

### Comparison of simulated and measured concentrations

To model the concentration curve, we fitted an analytical solution of the advection-dispersion-equation to the concentration data.
As a consistency if the model is a good approximation to the data, we want to compare the area under the simulated concentration curve to the area under the curve obtained with measurements.

In [ ]:
# You do not need to change anything in this cell

def advection_dispersion(t, x, v, D, m, A):
    """Compute the solution to the advection-dispersion equation"""
    def scalar(tt):
        """Piecewise definition for a single time point"""
        # For t=0, return the initial value
        if tt <= 0:
            return 0.0
        # Otherwise, compute the anaytical solution
        return (
            m / (A * v)
            * x / (np.sqrt(4 * np.pi * D * tt**3))
            * np.exp(-((x - v * tt) ** 2) / (4 * D * tt))
        ) / 1000 # convert from g/m³ to g/L

    f = np.vectorize(scalar, otypes=[float])
    return f(t)


# Fit the model to the data to obtain the parameters v, D and A
x = 20  # m
f = lambda time, v, D, A: advection_dispersion(time, x, v, D, mass, A)
popt, pcov = curve_fit(
    f, xdata=time, ydata=concentration, p0=(0.2, 1, 0.5), bounds=(0, np.inf)
)
v, D, area = popt  # units: m/s, m²/s, m²

# Plot the fitted curve along with the data
time_grid = np.linspace(0, time[-1], 100)
c_analytical = advection_dispersion(time_grid, x, v, D, mass, area)
plt.plot(time_grid, c_analytical, label="simulation")
plt.scatter(time, concentration, label="measurements")
plt.xlabel("time [s]")
plt.ylabel("concentration [g/L]")
plt.legend()



<div style="background-color:#AABAB2; color: black; width:90%; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px">
<p>

**Task 1.4**
    
Evaluate the integral of the simulated concentration curve from time 0 to 200 seconds, using Simpson's rule.

Vary the number of integration intervals from 2 to 50. Why does the number of integration intervals have to be an even number?

Observe how the result changes with different numbers of integration intervals.
</p>
</div>

In [ ]:
# Create an array of the number of integration intervals to test
intervals = np.arange(2, 50, step=2)
# Initialize a list to save the value of the integral for each number
#  of integration intervals
integrals = []
for n_intervals in intervals:
    # Compute the interval length
    t_end = time[-1]
    dt = t_end / n_intervals
    # Evaluate the function at the required time points
    t = np.linspace(0, t_end, n_intervals+1)
    f_evaluated = advection_dispersion(t, x, v, D, mass, area)
    # Integrate the concentration curve with Simpson's rule
    terms = [
        (f_evaluated[2 * i - 2] + 4 * f_evaluated[2 * i - 1] + f_evaluated[2 * i]) / 6 * 2 * dt
        for i in range(1, n_intervals // 2 + 1)
    ]
    integral = sum(terms)
    # Save the value of the integral in the list
    integrals.append(integral)

In [ ]:
# Create an array of the number of integration intervals to test
intervals = np.arange(### YOUR CODE HERE### )
# Initialize a list to save the value of the integral for each number
#  of integration intervals
integrals = []
for n_intervals in intervals:
    # Compute the interval length
    t_end = time[-1]
    dt = ### YOUR CODE HERE ###
    # Evaluate the function at the required time points
    t = np.linspace(0, t_end, n_intervals+1)
    f_evaluated = ### YOUR CODE HERE ###
    # Integrate the concentration curve with Simpson's rule
    integral = ### YOUR CODE LINES HERE ###
    # Save the value of the integral in the list
    integrals.append(integral)

In [ ]:
plt.plot(intervals, integrals, marker="o")
plt.xlabel("Nr. of integration intervals")
plt.ylabel("area under the curve");



<div style="background-color:#AABAB2; color: black; width:90%; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px">
<p>

**Task 1.5**
    
Compare the integral of the simulated concentrations with the integral of the measurements. Are the areas approximately the same?
</p>
</div>

<div style="background-color:#FAE99E; color: black; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px; width: 90%">
<p>

**Solution 1.5**

The numerical integral of the measured curve is approximately 34.8, whereas the integral of the simulated curve is approximately 32.8.
Thus, the mass captured by the simulation is close to, but slightly less than the mass captured through measureents.
One reason for this mismatch is that the simulated curve does not exactly fit the shape of the measured curve

</p>
</div>

## Part 2: Taylor series approximation

Taylor series are a powerful tool to approximate functions.
Throughout this week’s chapter in the text book, you have seen how Taylor series can be used to derive numerical approximations for derivatives and estimate the errors of numerical integration and differentiation techniques.

In the following exercies, we will have a closer look at how this approximation works.
The visualizations you create will hopefully help you to build some intuition for the meaning of a Taylor series.

We will approximate the natural logarithm by a number of Taylor polynomials of increasing order.

### Analytical expression

<div style="background-color:#AABAB2; color: black; width:90%; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px">
<p>

**Task 2.1**

This is a pen-and-paper exercise. Write down the terms first four terms of the Taylor series for $f(x) = \ln(x)$ around $x_0=1$.

</p>
</div>

<div style="background-color:#FAE99E; color: black; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px; width: 90%">
<p>

**Solution 2.1**

The first four derivatives of $f(x) = \ln(x)$ are:
$$f^\prime(x) = \frac{1}{x}$$
$$f^{\prime\prime}(x) = -\frac{1}{x^2}$$
$$f^{\prime\prime\prime}(x) = \frac{2}{x^3}$$
$$f^{\prime\prime\prime\prime}(x) = -\frac{6}{x^4}$$

In general,the $n$ th derivative is:
$$f^{(n)}(x) = (-1)^{n+1} (n-1)! \,x^{-n}$$

If we evaluate the derivative at $x_0=1$, we obtain:

$$f^\prime(1) = 1$$
$$f^{\prime\prime}(1) = -1$$
$$f^{\prime\prime\prime}(1) = 2$$
$$f^{\prime\prime\prime\prime}(1) = -6$$

The first four terms of the Taylor series around $x_0=1$ are:

$$
\begin{aligned}
f(x) &=  f(x_0) + (x-x_0)f'(x_0)+\frac{(x-x_0)^2}{2!}f''(x_0)+ \frac{(x-x_0)^3}{3!} f'''(x_0)+ \frac{(x-x_0)^4}{4!} f''''(x_0)+ ...\\
&= 0 + (x-1) \cdot 1 + \frac{(x-1)^2}{2!}\cdot (-1) + \frac{(x-1)^3}{3!}\cdot 2 + \frac{(x-1)^4}{4!}\cdot (-6) + \dots\\
&= (x-1) - \frac{(x-1)^2}{2} + \frac{(x-1)^3}{3} - \frac{(x-1)^4}{4} + \dots
\end{aligned}
$$

</p>
</div>

### Plotting the Taylor polynomials

<div style="background-color:#AABAB2; color: black; width:90%; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px">
<p>

**Task 2.2**

Before continuing with Taylor series approximation, plot the expression $f(x)=\ln(x)$ in the interval $[0, 3]$. This will be used as benchmark to assess your approximations. We will want to produce plots that will include each successive term of the Taylor approximation to see how the approximation improves as we include more terms in the Taylor series.

</p>
</div>

In [ ]:
x = np.linspace(### YOUR CODE HERE ###, num=200)

def f(x):
    return ### YOUR CODE HERE ###

plt.plot(x, f(x))
plt.xlabel('x')
plt.ylabel('f(x)')
plt.title("Plot of $f(x) = \\ln(x)$");

In [ ]:
x = np.linspace(0, 3, num=200)

def f(x):
    return np.log(x)

plt.plot(x, f(x))
plt.xlabel('x')
plt.ylabel('f(x)')
plt.title("Plot of $f(x) = \\ln(x)$");

<div style="background-color:#AABAB2; color: black; width:90%; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px">
<p>

**Task 2.3**

Implement a Python function that evaluats the nth derivative of $\ln(x)$ (at least up to $n=4$), so we can use it to compute the Talor polynomial.
Complete the function template in the code cell below.

Tip: to compute the factorial, you can use the function `math.factorial` from the Python standard library.

</p>
</div>

In [ ]:
def fn(x, n):
    """Compute the nth derivative of ln(x) at point x"""
    ### YOUR CODE (LINES) HERE
    return ### YOUR CODE HERE ###

In [ ]:
def fn(x, n):
    """Compute the nth derivative of ln(x) at point x"""
    return (-1)**(n+1) * math.factorial(n-1) * x**(-n)

<div style="background-color:#AABAB2; color: black; width:90%; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px">
<p>

**Task 2.4**

Define the expansion point $x_0$ in Python and provide Python expressions for the Taylor polynomials of first, second, third and fourth order.

</p>
</div>

In [ ]:
x0 = # YOUR_CODE_HERE
taylor_1 = # YOUR_CODE_HERE
taylor_2 = # YOUR_CODE_HERE
taylor_3 = # YOUR_CODE_HERE
taylor_4 = # YOUR_CODE_HERE

In [ ]:
x0 = 1
taylor_1 = f(x0) + fn(x0, 1) * (x - x0)
taylor_2 = taylor_1 + fn(x0, 2) * (x - x0) ** 2 / math.factorial(2)
taylor_3 = taylor_2 + fn(x0, 3) * (x - x0) ** 3 / math.factorial(3)
taylor_4 = taylor_3 + fn(x0, 4) * (x - x0) ** 4 / math.factorial(4)

<div style="background-color:#AABAB2; color: black; width:90%; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px">
<p>

**Task 2.5**

Fill in the code below to plot the function $f(x)$ along with the Taylor polynomials of increasing order.

</p>
</div>

In [ ]:
# Plot f(x) and the Taylor polynomials
plt.plot(x, f(x), label="$f(x) = \\ln(x)$", color="k")
plt.plot(x, ### YOUR CODE HERE ###, label="First Order", color="C0")
plt.plot(x, ### YOUR CODE HERE ###, label="Second Order", color="C1")
plt.plot(x, ### YOUR CODE HERE ###, label="Third Order", color="C2")
plt.plot(x, ### YOUR CODE HERE ###, label="Fourth Order", color="C3")

# Plot the expansion point x0 as a dot
plt.scatter(
    ### YOUR CODE HERE ###,
    ### YOUR CODE HERE ###,
    color="k",
    marker="o",
    label=f"Expansion Point ($x = {x0:0.2f}$)"
)

# Add labels to the plot
plt.xlabel("x")
plt.ylabel("$f(x)$")
plt.title("Taylor polynomials of $f(x) = \\ln(x)$")
plt.legend()

In [ ]:
# Plot f(x) and the Taylor polynomials
plt.plot(x, f(x), label="$f(x) = \\ln(x)$", color="k")
plt.plot(x, taylor_1, label="First Order", color="C0")
plt.plot(x, taylor_2, label="Second Order", color="C1")
plt.plot(x, taylor_3, label="Third Order", color="C2")
plt.plot(x, taylor_4, label="Fourth Order", color="C3")

# Plot the expansion point x0 as a dot
plt.scatter(
    [x0], [f(x0)], color="k", marker="o", label=f"Expansion Point ($x = {x0:0.2f}$)"
)

# Add labels to the plot
plt.xlabel("x")
plt.ylabel("$f(x)$")
plt.title("Taylor polynomials of $f(x) = \\ln(x)$")
plt.legend()

<div style="background-color:#AABAB2; color: black; width:90%; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px">
<p>

**Task 2.6**

Look at the plot you just created. Which Taylor polynomial approximates the function $f(x)$ the best? How does depend on the distance from the expansion point $x_0$?

</p>
</div>


### Absolute error

To further analyze the error introduced by truncating the Taylor series, we can evaluate the absolute difference between the function $f(x)$ and the Taylor polynomials:

$$\text{error} =|f(x)-T_n|\,,$$

where $T_n$ refers to the Taylor polynomial of $n$th order.

<div style="background-color:#AABAB2; color: black; width:90%; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px">
<p>

**Task 2.7**

Use your Taylor series approximations and the analytic expression for $f(x)$ to determine the absolute error. Plot the error against $x$ and vary the $x$- and $y$-limits. Are the larger orders Taylor polynomials always more accurate? 

</p>
</div>

In [ ]:
# Compute the absolute error
error_1 = ### YOUR CODE HERE ###
error_2 = ### YOUR CODE HERE ###
error_3 = ### YOUR CODE HERE ###
error_4 = ### YOUR CODE HERE ###

# Plot the absolute error against x
plt.plot(x, ### YOUR CODE HERE ###, label="First Order", color="C0")
plt.plot(x, ### YOUR CODE HERE ###, label="Second Order", color="C1")
plt.plot(x, ### YOUR CODE HERE ###, label="Third Order", color="C2")
plt.plot(x, ### YOUR CODE HERE ###, label="Fourth Order", color="C3")

# Add labels
plt.xlabel("x")
plt.ylabel("Absolute Error: $f(x)-\\mathrm{Taylor~polynomial}$")
plt.title("Absolute Error of Taylor Series Approximations")
plt.legend()

In [ ]:
# Compute the absolute error
error_1 = np.abs(f(x) - taylor_1)
error_2 = np.abs(f(x) - taylor_2)
error_3 = np.abs(f(x) - taylor_3)
error_4 = np.abs(f(x) - taylor_4)

# Plot the absolute error against x
plt.plot(x, error_1, label="First Order", color="C0")
plt.plot(x, error_2, label="Second Order", color="C1")
plt.plot(x, error_3, label="Third Order", color="C2")
plt.plot(x, error_4, label="Fourth Order", color="C3")

# Add labels
plt.xlabel("x")
plt.ylabel("Absolute Error: $f(x)-\\mathrm{Taylor~polynomial}$")
plt.title("Absolute Error of Taylor Series Approximations")
plt.legend()

## Part 3: Deriving numerical derivatives from Taylor series expansions

After we had a closer look at the meaning of Taylor series, you will now practice how you can apply Taylor series to derive expressions for numerical derivatives.

<div style="background-color:#AABAB2; color: black; width:90%; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px">
<p>

**Task 3.1**
    
Derive the backward difference method such that it is *second order* accurate. Refer to the book for an illustration of this approach with first order accuracy. Insert an image of your math below.

*You don't have to further use the result in this assignment, but it is useful to understand the approach and prepare for the exam.*

Tips:
- You will have to combine Taylor series expansions for the function at two different points.
- At what order do you need to truncate the Taylor series to get a second order accurate method?    
</p>
</div>

<div style="background-color:#FAE99E; color: black; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px; width: 90%">
<p>

**Solution 3.1**
    
For backward differences we evaluate the Taylor series around $x_i$ at $x_{i−1}$ and $x_{i−2}$.
To end up with a second-order accurate expression, we need to inlude terms at least up to $n=2$.
This yields the following Taylor approximations:
    
$$f(x_{i-1})\approx f(x_{i})+(x_{i-1}-x_i)\frac{\partial f(x_{i})}{\partial x} +\frac{(x_{i-1}-x_i)^2}{2}\frac{\partial^2 f(x_i)}{\partial x^2}+\mathcal{O}(x_{i-1}-x_i)^3$$
$$f(x_{i-2})\approx f(x_{i})+(x_{i-2}-x_i)\frac{\partial f(x_{i})}{\partial x} +\frac{(x_{i-2}-x_i)^2}{2}\frac{\partial^2 f(x_i)}{\partial x^2}+\mathcal{O}(x_{i-2}-x_i)^3$$

    
We set $\Delta x = x_i - x_{i-1}$ for all $i$, which also means: $2\Delta x = x_i-x_{i-2}$
    
$$f(x_{i-1})\approx f(x_{i})-\Delta x\frac{\partial f(x_{i})}{\partial x} +\frac{\Delta x^2}{2}\frac{\partial^2 f(x_i)}{\partial x^2}+\mathcal{O}(\Delta x)^3$$
$$f(x_{i-2})\approx f(x_{i})-2\Delta x\frac{\partial f(x_{i})}{\partial x} +\frac{4\Delta x^2}{2}\frac{\partial^2 f(x_i)}{\partial x^2}+\mathcal{O}(\Delta x)^3$$
    
To get rid of the term including the second derivative, we multiply the first expression by 4 and subtract the second expression:
    
$$
\begin{aligned}
4f(x_{i-1})-f(x_{i-2})&\approx (4-1)f(x_{i})-(4-2)\Delta x\frac{\partial f(x_{i})}{\partial x} + (4-4)\frac{\Delta x^2}{2}\frac{\partial^2 f(x_i)}{\partial x^2}+\mathcal{O}(\Delta x)^3\\
&= 3f(x_{i})-2\Delta x\frac{\partial f(x_{i})}{\partial x} + \mathcal{O}(\Delta x)^3
\end{aligned}
$$

Bring the derivative to the left side and all terms involving $f(x)$ to the right side:
$$ 2\Delta x\frac{\partial f(x_{i})}{\partial x} \approx 3f(x_i)-4f(x_{i-1})+f(x_{i-2}) +\mathcal{O}(\Delta x)^3$$

Divide by $2 \Delta x$:

$$\frac{\partial f(x_{i})}{\partial x} \approx \frac{3f(x_i)-4f(x_{i-1})+f(x_{i-2})}{2\Delta x} +\mathcal{O}(\Delta x)^2$$

Note how the order of the error changes because we divide by $\Delta x$.
The final expression is second-order accurate.

</p></div>

<div style="margin-top: 50px; padding-top: 20px; border-top: 1px solid #ccc;">
  <div style="display: flex; justify-content: flex-end; gap: 20px; align-items: center;">
    <a rel="MUDE" href="http://mude.citg.tudelft.nl/">
      <img alt="MUDE" style="width:100px; height:auto;" src="https://gitlab.tudelft.nl/mude/public/-/raw/main/mude-logo/MUDE_Logo-small.png" />
    </a>
    <a rel="TU Delft" href="https://www.tudelft.nl/en/ceg">
      <img alt="TU Delft" style="width:100px; height:auto;" src="https://gitlab.tudelft.nl/mude/public/-/raw/main/tu-logo/TU_P1_full-color.png" />
    </a>
    <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">
      <img alt="Creative Commons License" style="width:88px; height:auto;" src="https://i.creativecommons.org/l/by/4.0/88x31.png" />
    </a>
  </div>
  <div style="font-size: 75%; margin-top: 10px; text-align: right;">
    &copy; Copyright 2025 <a rel="MUDE" href="http://mude.citg.tudelft.nl/">MUDE</a> TU Delft. 
    This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">CC BY 4.0 License</a>.
  </div>
</div>